In [1]:
import pandas as pd

In [2]:
# base json file needs to be encased on [] and have a comma separating each item
# modifies the json file to be readable. into business.json

import os.path

def reformatJson():
    with open('data/meta-California.json', 'r') as infile, open('data/business.json', 'w') as outfile:
        outfile.write('[\n')
        lines = infile.readlines()

        # adds comma to every line except last
        for i, line in enumerate(lines):
            if i < len(lines) - 1: line = line.rstrip() + ',\n'
            outfile.write(line)

        outfile.write(']')
        
if not os.path.isfile('data/business.json') : reformatJson()

In [3]:
cali = pd.read_json('data/business.json')

In [4]:
dropped = cali.drop(columns=['gmap_id', 'description', 'MISC', 'state', 'url', 'relative_results'])
dropped = dropped.drop(columns=['price']) # for now
dropped = dropped.dropna()
dropped.columns.tolist()

['name',
 'address',
 'latitude',
 'longitude',
 'category',
 'avg_rating',
 'num_of_reviews',
 'hours']

In [14]:
target_cities = ["Davis", "Sacramento"]

# is the address in the target cities list
def city_filter (row) :
    try: return row.address.split(', ')[2] in target_cities
    except: return False

df = dropped
city_only = df[df.apply(city_filter, axis=1)]

print(city_only.size)
city_only.head()

75264


,name,address,latitude,longitude,category,avg_rating,num_of_reviews,hours
13,Aurora Massage,"Aurora Massage, 2033 Marconi Ave, Sacramento, ...",38.619557,-121.417868,[Massage therapist],4.2,56,"[[Thursday, 9:30AM–9:30PM], [Friday, 9:30AM–9:..."
15,Orient Massage,"Orient Massage, 5800 Madison Ave # U2, Sacrame...",38.659634,-121.327812,[Massage therapist],3.6,15,"[[Thursday, 10AM–8PM], [Friday, 10AM–8PM], [Sa..."
18,Porvene Doors,"Porvene Doors, 4119 S Market Ct # D, Sacrament...",38.644575,-121.483406,[Door supplier],4.0,4,"[[Thursday, 7AM–4PM], [Friday, 7AM–4PM], [Satu..."
23,Origin Church,"Origin Church, 4600 Winding Way, Sacramento, C...",38.642165,-121.356162,[Church],5.0,8,"[[Thursday, Closed], [Friday, Closed], [Saturd..."
141,River City Staffing,"River City Staffing, 3301 Watt Ave #100, Sacra...",38.625492,-121.383591,[Employment agency],3.2,13,"[[Thursday, 8AM–5PM], [Friday, 8AM–5PM], [Satu..."


In [6]:
import re

def string_to_hours (string : str, AM = False) -> (float, bool):
    match = re.match(r'(\d+)(?::(\d+))?(\D*)', string)
    hours, minutes, period = match.groups()
    AM = (period == '' and AM) or period == "AM"
    return (float(hours) + (0 if AM else 12) + (float(minutes) if minutes else 0) / 60, AM)

def range_to_hours (string: str) -> float:
    if string == 'Open 24 hours': return 24
    elif string == 'Closed' : return 0
    try:
        start, end = string.split('–')
        endTime, endPeriod = string_to_hours(end)
        startTime = string_to_hours(start, endPeriod)[0]
        difference = endTime - startTime
        return difference if difference > 0 else 24 + difference
    except:
        raise Exception("Couldn't enumerate: " + string)

weekly_hours = dropped.hours.map(\
    lambda entry: sum(map(\
        lambda day: range_to_hours(day[1]), entry)\
    )\
)

In [7]:
modified = city_only.copy()
modified = modified.drop(columns=['hours'])
modified['weekly_hours'] = weekly_hours
modified.head()

,name,address,latitude,longitude,category,avg_rating,num_of_reviews,weekly_hours
13,Aurora Massage,"Aurora Massage, 2033 Marconi Ave, Sacramento, ...",38.619557,-121.417868,[Massage therapist],4.2,56,84.0
15,Orient Massage,"Orient Massage, 5800 Madison Ave # U2, Sacrame...",38.659634,-121.327812,[Massage therapist],3.6,15,70.0
18,Porvene Doors,"Porvene Doors, 4119 S Market Ct # D, Sacrament...",38.644575,-121.483406,[Door supplier],4.0,4,45.0
23,Origin Church,"Origin Church, 4600 Winding Way, Sacramento, C...",38.642165,-121.356162,[Church],5.0,8,14.0
141,River City Staffing,"River City Staffing, 3301 Watt Ave #100, Sacra...",38.625492,-121.383591,[Employment agency],3.2,13,45.0


In [8]:
city_only.category

13                                      [Massage therapist]
15                                      [Massage therapist]
18                                          [Door supplier]
23                                                 [Church]
141                                     [Employment agency]
                                ...                        
515756                       [Ice cream shop, Dessert shop]
515763                                              [Store]
515777    [Ice cream shop, Dessert restaurant, Donut sho...
515790                                     [Animal shelter]
515802                                          [Bank, ATM]
Name: category, Length: 9408, dtype: object